In [3]:
import re
import os
import csv
import time
import shutil
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [12]:
Registers = []
with open("C:/Analytics/Lista_de_Ruc2.csv", mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        Registers.append(row[0])  # Suponiendo que el dato está en la primera columna
Registers

['1790300404001',
 '0891742622001',
 '1390012949001',
 '1391794495001',
 '1391932573001']

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()
wait = WebDriverWait(driver, 10)
fecha_consulta = datetime.now()
for register in Registers:
    driver.get("https://permisosfuncionamiento.controlsanitario.gob.ec/permisosdefuncionamiento/index.php")
    time.sleep(2)
    boton_ruc = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div/form/div[2]/div/div/center[2]/table/tbody/tr[2]/td/center/table/tbody/tr[4]/td/div/div[2]/input")))
    boton_ruc.click()
    time.sleep(2)
    input_element = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div/form/div[2]/div/div/center[2]/table/tbody/tr[2]/td/center/table/tbody/tr[2]/td/div/div[2]/input")))
    # Ingresar el número en el campo de texto
    input_element.send_keys(register)
    time.sleep(3)
    #Click en boton consultar
    elemento_xpath = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div/form/div[2]/div/div/center[2]/table/tbody/tr[3]/td/div/div/button")))
    elemento_xpath.click()
    time.sleep(3)
    elemento_xpath = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div/form/div[2]/div/div/center[2]/div/div[1]/div/div[2]/div/section/div/div/div/div/div/div/div[1]/button[2]")))
    elemento_xpath.click()
    time.sleep(3)
driver.quit()

#rutas de archivos
downloads_folder = os.path.expanduser("C:/Users/eeras/Downloads")  # Ajusta TU_USUARIO
destination_folder = "C:/Analytics/Arcsa/excel"
excel_path = "C:/Analytics/Arcsa/permisos.xlsx"

pattern = re.compile(r"Consulta Permisos de Funcionamiento(?: \(\d+\))?\.xlsx$")

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

for file in os.listdir(downloads_folder):
    if file.endswith(".xlsx") and pattern.match(file):  
        src_path = os.path.join(downloads_folder, file)
        dest_path = os.path.join(destination_folder, file)
        shutil.move(src_path, dest_path)
        print(f"Movido: {file}")
data_list = []
fecha_consulta = pd.to_datetime(datetime.now().date())  
for file in os.listdir(destination_folder):
    if file.endswith(".xlsx"):
        file_path = os.path.join(destination_folder, file)
        df = pd.read_excel(file_path, skiprows=1)
        #print("Columnas encontradas:", df.columns.tolist())  
        #df.columns = df.columns.str.strip()

        df = df[["Número de Permiso", "Fecha de Emisión de Permiso", "Fecha de Vigencia de Permiso", "RUC", "Razón Social"]]
        df["Fecha de Emisión de Permiso"] = pd.to_datetime(df["Fecha de Emisión de Permiso"],dayfirst=True, errors="coerce")
        df["Fecha de Vigencia de Permiso"] = pd.to_datetime(df["Fecha de Vigencia de Permiso"],dayfirst=True, errors="coerce")

        if df["Fecha de Vigencia de Permiso"].isnull().sum() > 0:
            print("Advertencia: Algunas fechas no pudieron convertirse correctamente.")

        # Calcular días para caducar permiso
        df["Tiempo en que caduca permiso"] = (df["Fecha de Vigencia de Permiso"] - fecha_consulta).dt.days
        df["Tiempo en que caduca permiso"] = df["Tiempo en que caduca permiso"].apply(
             lambda x: f"Faltan {x} días para que caduque el permiso" if pd.notna(x) and x >= 0 else "Caducado")

        df["Días restantes"] = df["Tiempo en que caduca permiso"].apply(
            lambda x: int(x.split()[1]) if "Faltan" in x else 0)

        # Definir estado
        df["Estado"] = df["Días restantes"].apply(
            lambda x: "Caducado" if x < 0 else 
            "Próximo a caducar" if x < 14 else "Está vigente")

        # Eliminar la columna auxiliar "Días restantes"
        df.drop(columns=["Días restantes"], inplace=True)
        
        data_list.append(df)
        
if data_list:
    final_df = pd.concat(data_list, ignore_index=True)
    final_df.to_excel(excel_path, index=False)
    print(f"Archivo consolidado guardado en {excel_path}")


Movido: Consulta Permisos de Funcionamiento (1).xlsx
Movido: Consulta Permisos de Funcionamiento (2).xlsx
Movido: Consulta Permisos de Funcionamiento (3).xlsx
Movido: Consulta Permisos de Funcionamiento (4).xlsx
Movido: Consulta Permisos de Funcionamiento.xlsx
Archivo consolidado guardado en C:/Analytics/Arcsa/permisos.xlsx
